In [1]:
import requests
import base64
import json
import logging
import os
import sys
import pytz
import warnings
import gc
import pandas as pd
from pathlib import Path
from datetime import datetime

warnings.filterwarnings("ignore")

base_url = 'https://jira.footlocker.com/rest/api/2/search'

In [84]:

def get_report(base_url,api_token):
    logger.info("Getting report results...")
    header_gs = {'Accept': 'application/json'}
    headers = { 'Authorization' : 'Bearer %s' %  api_token}
    # 'Content-Length':3000 }
    logger.info(base_url)
    
    for i in range(0,3):
        r = requests.get(base_url,headers=headers)
        logger.info(r)
        if r.ok:
            logger.info("Report results received...")
            logger.info("HTTP %i - %s" % (r.status_code, r.reason))
            return r.text
        else:
            logger.error("HTTP %i - %s" % (r.status_code, r.reason))
            continue
    return None

In [91]:
def getDataSet(query, project, issueType,api_token):
    pd_obj = None
    maxRslt = 500
    totalRsltFetch = -1
    df= pd.DataFrame()
    while True:
        dataset= get_report(base_url+query+f"&maxResults={maxRslt}&startAt={totalRsltFetch + 1 }", api_token)
        if dataset != None:
            obj = json.loads(dataset)
            logger.info(f"total: {obj['total']} ({project} - {issueType})")
            totalRsltFetch = totalRsltFetch + maxRslt

            if obj['issues']:
                df = pd.concat([df, pd.json_normalize(obj['issues']) ], ignore_index=True)
            else:
               logger.error(f'No records found ({project} - {issueType})....')
            if (totalRsltFetch >= obj['total'] ):
                break
        # pd_obj = pd.json_normalize(obj['issues'])
        else:
            logger.error(f'Dataset was empty ({project} - {issueType})....')
            break
        # break       

    return df
    

In [86]:
def getUrlDataSet(url,api_token):
    pd_obj = None

    dataset= get_report(ur, api_token)
    if dataset != None:
        pd_obj = json.loads(dataset)
        # pd_obj = pd.json_normalize(obj['issues'])
    return pd_obj

In [87]:
def setup_logger( level=logging.WARNING):
    """Configures a logger and returns it."""

    # if not os.path.exists('./log'):
    #     os.makedirs('./log')

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # handler = logging.FileHandler('./log/soleProj.log')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
   
    logger = logging.getLogger()
    logger.setLevel(level)
    
    if (logger.hasHandlers()):
        logger.handlers.clear()

    logger.addHandler(handler)
   

    return logger

In [ ]:
def projectList():
    searchURL='https://jira.footlocker.com/rest/api/2/project'
    searchQry_DS = getUrlDataSet(searchURL)
    df=pd.json_normalize(searchQry_DS)
    df=df.drop(df[df['key'].isin(['DEVOPS','EPL'])].index)
    return df['key']

In [88]:

def sprintvalue(x,field):
    if(len(x)!=0):
      return x[field]
    else:
     return 'NA'

In [89]:
def keyvalue(x,field):
    if(len(x)!=0):
      return x[0][field]
    else:
     return 'NA'


In [90]:
def clean_folder(filepath,finder):
    # Get all .csv files in the 'data' directory
    csv_files = Path(filepath).glob(f"*{finder}")                                   
    for file in csv_files:
        filename=file.name
        newname= filename.replace(finder,'.csv')
        logger.warning(f'renaming {filename} TO {newname}....')
        os.rename(os.path.join(filepath, filename), os.path.join(filepath, newname))
        os.utime(os.path.join(filepath, newname), None)